In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('transactions-pet_store-small.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              300 non-null    object 
 1   Order_Number      300 non-null    object 
 2   Customer_ID       288 non-null    object 
 3   Product_Name      271 non-null    object 
 4   SKU               300 non-null    object 
 5   Price             300 non-null    float64
 6   Size              71 non-null     object 
 7   Quantity          300 non-null    int64  
 8   Product_Category  155 non-null    object 
 9   Product_Line      298 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 23.6+ KB


,Date,Order_Number,Customer_ID,Product_Name,SKU,Price,Size,Quantity,Product_Category,Product_Line
0,3/31/2019,5ZR-4930-9569-1000,e2b695c8-bf79-46d5-9455-f53f68562af2,NaN,RKAPY3I1TP,39.55,NaN,2,NaN,NaN
1,3/8/2020,TAI-6046-9953-7908,NaN,NaN,O5FYJLBE0H,24.53,NaN,2,NaN,NaN
2,3/13/2021,KZM-2608-1918-3044,4c3aa8dc-fc7b-4796-9c70-b0aa9a4f1977,NaN,I2GQUNYDXW,36.30,medium,2,NaN,dog
3,6/30/2019,VXJ-7724-4518-8390,6cb2df70-b4e9-41c6-8b90-a375376c55dd,NaN,W86BRJ9SSG,39.32,NaN,3,NaN,cat
4,5/21/2019,5GV-6876-7635-4506,9d92f453-84ec-4beb-acc1-2b9d1cce8910,NaN,CG3531YP08,38.17,NaN,2,NaN,dog


In [3]:
# Checking numeric values for obvious errors

df.describe()

,Price,Quantity
count,300.000000,300.000000
mean,26.032167,2.103333
std,8.939657,0.792517
min,10.800000,1.000000
25%,18.530000,1.000000
50%,25.480000,2.000000
75%,34.380000,3.000000
max,39.550000,3.000000


In [4]:
# Checking a few SKUs to decide use them for getting missing values in another columns
# Here I found that I can find missing Product_Name, Product_Category and Product_Line using SKU

# df[df.SKU == 'RKAPY3I1TP']
# df[df.SKU == 'O5FYJLBE0H']
df[df.SKU == 'W86BRJ9SSG'].head()

,Date,Order_Number,Customer_ID,Product_Name,SKU,Price,Size,Quantity,Product_Category,Product_Line
3,6/30/2019,VXJ-7724-4518-8390,6cb2df70-b4e9-41c6-8b90-a375376c55dd,NaN,W86BRJ9SSG,39.32,NaN,3,NaN,cat
8,7/14/2021,3RH-2068-8836-6576,6909526c-22bc-4feb-9bac-d381236876ff,NaN,W86BRJ9SSG,39.32,NaN,2,NaN,cat
65,7/13/2021,2VI-3304-1801-5314,b3c6c8a8-6db2-4920-8b5c-0c889c60b1eb,Kitty Climber,W86BRJ9SSG,39.32,NaN,2,NaN,cat
72,1/4/2020,UIR-2760-1217-0744,9661b347-470c-4a8b-8924-fda98c4e935c,Kitty Climber,W86BRJ9SSG,39.32,NaN,2,NaN,cat
76,7/22/2020,XD9-1197-7351-9457,041686b3-ef2e-4ae9-a349-985bd6dafed3,Kitty Climber,W86BRJ9SSG,39.32,NaN,2,NaN,cat


In [5]:
# Checking how many unique products we have

print(len(df.SKU.unique()))

23


In [6]:
# Creating a reference table for SKU and the corresponding non-missing values

reference_table = df.dropna(subset=['SKU', 'Product_Name', 'Product_Category', 'Product_Line']).drop_duplicates(subset='SKU')
reference_table = reference_table[['SKU', 'Product_Name', 'Product_Category', 'Product_Line']]

# Defining a function to fill missing values from the reference table

def fill_missing_values(row, ref_table, column_name):
    if pd.isnull(row[column_name]):
        ref_value = ref_table[ref_table['SKU'] == row['SKU']][column_name]
        if not ref_value.empty:
            return ref_value.iloc[0]
    return row[column_name]

# Fill in missing values in df using the reference table

for column in ['Product_Name', 'Product_Category', 'Product_Line']:
    df[column] = df.apply(fill_missing_values, args=(reference_table, column), axis=1)

df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              300 non-null    object 
 1   Order_Number      300 non-null    object 
 2   Customer_ID       288 non-null    object 
 3   Product_Name      300 non-null    object 
 4   SKU               300 non-null    object 
 5   Price             300 non-null    float64
 6   Size              71 non-null     object 
 7   Quantity          300 non-null    int64  
 8   Product_Category  300 non-null    object 
 9   Product_Line      300 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 23.6+ KB


,Date,Order_Number,Customer_ID,Product_Name,SKU,Price,Size,Quantity,Product_Category,Product_Line
0,3/31/2019,5ZR-4930-9569-1000,e2b695c8-bf79-46d5-9455-f53f68562af2,Feline Fix Mix,RKAPY3I1TP,39.55,NaN,2,treat,cat
1,3/8/2020,TAI-6046-9953-7908,NaN,Purr Mix,O5FYJLBE0H,24.53,NaN,2,food,cat
2,3/13/2021,KZM-2608-1918-3044,4c3aa8dc-fc7b-4796-9c70-b0aa9a4f1977,Reddy Beddy,I2GQUNYDXW,36.30,medium,2,bedding,dog
3,6/30/2019,VXJ-7724-4518-8390,6cb2df70-b4e9-41c6-8b90-a375376c55dd,Kitty Climber,W86BRJ9SSG,39.32,NaN,3,toy,cat
4,5/21/2019,5GV-6876-7635-4506,9d92f453-84ec-4beb-acc1-2b9d1cce8910,Chewie Dental,CG3531YP08,38.17,NaN,2,treat,dog


In [7]:
# I assumed if Size NaN, it means the product doesn’t have different sizes
# Replacing all NaN values in Size column to 'one size'

df['Size'].fillna('one size', inplace=True)
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Date              300 non-null    object 
 1   Order_Number      300 non-null    object 
 2   Customer_ID       288 non-null    object 
 3   Product_Name      300 non-null    object 
 4   SKU               300 non-null    object 
 5   Price             300 non-null    float64
 6   Size              300 non-null    object 
 7   Quantity          300 non-null    int64  
 8   Product_Category  300 non-null    object 
 9   Product_Line      300 non-null    object 
dtypes: float64(1), int64(1), object(8)
memory usage: 23.6+ KB


,Date,Order_Number,Customer_ID,Product_Name,SKU,Price,Size,Quantity,Product_Category,Product_Line
0,3/31/2019,5ZR-4930-9569-1000,e2b695c8-bf79-46d5-9455-f53f68562af2,Feline Fix Mix,RKAPY3I1TP,39.55,one size,2,treat,cat
1,3/8/2020,TAI-6046-9953-7908,NaN,Purr Mix,O5FYJLBE0H,24.53,one size,2,food,cat
2,3/13/2021,KZM-2608-1918-3044,4c3aa8dc-fc7b-4796-9c70-b0aa9a4f1977,Reddy Beddy,I2GQUNYDXW,36.30,medium,2,bedding,dog
3,6/30/2019,VXJ-7724-4518-8390,6cb2df70-b4e9-41c6-8b90-a375376c55dd,Kitty Climber,W86BRJ9SSG,39.32,one size,3,toy,cat
4,5/21/2019,5GV-6876-7635-4506,9d92f453-84ec-4beb-acc1-2b9d1cce8910,Chewie Dental,CG3531YP08,38.17,one size,2,treat,dog


In [8]:
# We also have some NaN values in Customer_ID column
# Here I found that I can find some missing Customer_ID values using Order_Number, so next, I'll try to replace them using the same methos as above

# df[df.Customer_ID.isna()]
df[df.Order_Number == '2OI-1191-6735-8296']

,Date,Order_Number,Customer_ID,Product_Name,SKU,Price,Size,Quantity,Product_Category,Product_Line
28,7/19/2020,2OI-1191-6735-8296,98395aed-edab-450a-a45b-df382ffbaecb,Snoozer Hammock,V4B4RNS3ZP,34.38,one size,2,bedding,cat
252,7/19/2020,2OI-1191-6735-8296,NaN,Scratchy Post,MPH6SCD7UT,26.95,one size,1,toy,cat


In [9]:
# Creating a reference table for Order_Number and the corresponding non-missing Customer_IDs

new_reference_table = df.dropna(subset=['Order_Number', 'Customer_ID']).drop_duplicates(subset='Order_Number')[['Order_Number', 'Customer_ID']]

# Modifying the function to fill missing values in Order_Number from the reference table

def fill_missing_id_values(row, ref_table, column_name):
    if pd.isnull(row[column_name]):
        ref_value = ref_table[ref_table['Order_Number'] == row['Order_Number']][column_name]
        if not ref_value.empty:
            return ref_value.iloc[0]
    return row[column_name]

# Assuming 'Customer_ID' and 'Order_Number' are the column names in your DataFrame

df['Customer_ID'] = df.apply(fill_missing_id_values, args=(new_reference_table, 'Customer_ID'), axis=1)

In [10]:
# We still have 8 rows with NaN Customer_ID and I decided to leave it at that for now

df[df.Customer_ID.isna()]

,Date,Order_Number,Customer_ID,Product_Name,SKU,Price,Size,Quantity,Product_Category,Product_Line
45,12/27/2020,VWZ-7707-5242-9683,NaN,Ball and String,1W0N5O3XOD,16.71,one size,3,toy,cat
46,4/18/2019,O3X-7590-8873-7538,NaN,New Dish,NYW2F6CPBY,17.16,large,3,food,dog
69,4/21/2021,KEC-4821-3488-4229,NaN,Ball and String,1W0N5O3XOD,16.71,one size,3,toy,cat
75,6/3/2020,RU5-3005-3729-1797,NaN,Fetch Blaster,M291KHJ4LW,29.47,one size,3,toy,dog
93,5/18/2019,TZ0-9472-4859-1531,NaN,Purr Mix,O5FYJLBE0H,24.53,one size,2,food,cat
117,7/8/2019,QXS-3817-6803-3259,NaN,Snoozer Essentails,GABWVMEL2R,28.04,one size,1,bedding,dog
237,5/1/2020,VXO-8072-6757-0780,NaN,Yum Fish-Dish,GZCJZ3ET04,30.63,one size,1,food,cat
262,4/15/2021,X5D-2338-5958-0974,NaN,Reddy Beddy,I2GQUNYDXW,36.30,medium,3,bedding,dog


In [11]:
# I also want to convert the Date column to datetime and add the Year_Month column to make it easier to work with while exploring

df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')
df['Year_Month'] = df['Date'].dt.to_period('M')
df.head()

,Date,Order_Number,Customer_ID,Product_Name,SKU,Price,Size,Quantity,Product_Category,Product_Line,Year_Month
0,2019-03-31,5ZR-4930-9569-1000,e2b695c8-bf79-46d5-9455-f53f68562af2,Feline Fix Mix,RKAPY3I1TP,39.55,one size,2,treat,cat,2019-03
1,2020-03-08,TAI-6046-9953-7908,7f5111f6-53c9-4316-b134-dd8c3b6b29f7,Purr Mix,O5FYJLBE0H,24.53,one size,2,food,cat,2020-03
2,2021-03-13,KZM-2608-1918-3044,4c3aa8dc-fc7b-4796-9c70-b0aa9a4f1977,Reddy Beddy,I2GQUNYDXW,36.30,medium,2,bedding,dog,2021-03
3,2019-06-30,VXJ-7724-4518-8390,6cb2df70-b4e9-41c6-8b90-a375376c55dd,Kitty Climber,W86BRJ9SSG,39.32,one size,3,toy,cat,2019-06
4,2019-05-21,5GV-6876-7635-4506,9d92f453-84ec-4beb-acc1-2b9d1cce8910,Chewie Dental,CG3531YP08,38.17,one size,2,treat,dog,2019-05


In [12]:
# Saving the filled dataset

df.to_csv('transactions-pet_store-small_scrubbed.csv', index=False)